In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader, random_split

from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import os 

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print (DEVICE)

cuda


In [2]:
!ls

A2_CNNs.ipynb	 q2_cnns.ipynb	  time_compare.ipynb	 vgg16
AlexNet		 Question1.ipynb  tiny-imagenet-200
health_data.csv  README.md	  tiny-imagenet-200.zip
LeNet		 testsmo.py	  utils


In [3]:
#let us load the data and all!
BATCH_SIZE = 64
IMAGE_DIR = './tiny-imagenet-200/train/'
leNet_transforms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
data = datasets.ImageFolder(IMAGE_DIR, transform = leNet_transforms)

train_data, val_data = random_split (data, [90000, 10000])
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=True)

In [4]:
%reload_ext autoreload
%autoreload 1
%aimport LeNet

In [5]:
%aimport LeNet.model
from vgg16.model import VGG16
N_CLASSES = 200
lr=1e-3

model = VGG16(N_CLASSES)
#using Adam Optimizer
optim_adam = torch.optim.Adam(model.parameters(), lr=lr)
lossfn = nn.CrossEntropyLoss()


In [6]:
from datetime import datetime 

def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='r', label='Training loss') 
    ax.plot(valid_losses, color='b', label='Testing loss')
    ax.set(title="Loss across epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default

def train_optm (train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)
    
        # Forward pass
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

def validate(valid_loader, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
        
    return model, epoch_loss

def training_loop_optm (model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
 
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train_optm (train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, valid_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)}     '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.3f}\t'
                  f'Test loss: {valid_loss:.3f}\t'
                  f'Train accuracy: {100 * train_acc:.3f}\t'
                  f'Test accuracy: {100 * valid_acc:.3f}')
            
    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)



In [ ]:
model = model.to(DEVICE)
model, optim_adam, losses = training_loop_optm(model, lossfn, optim_adam, train_loader, val_loader, 25, DEVICE, print_every=1)